# **Part Two of the Course Project**
In this project, you will encode movie descriptions into numeric vectors and cluster them using various hyperparameters, i.e., parameters that you will specify prior to model training. You will also search for the best hierarchical clustering ([HC](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html)) scenario based on its silhouette score.
<hr style="border-top: 2px solid #606366; background: transparent;">

# **Setup**
 
Reset the Python environment to clear it of any previously loaded variables, functions, or libraries. Then, import the libraries and The Movie Database ([TMDB](https://www.themoviedb.org/)) compressed file needed for this project.

In [ ]:
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS
IS.ast_node_interactivity = "all"    # allows multiple outputs from a cell
import pandas as pd, numpy as np, nltk, matplotlib.pyplot as plt, seaborn as sns
import numpy.testing as npt, unittest
from sklearn.cluster import AgglomerativeClustering as HC  # hierarchical clustering
from sklearn.metrics import adjusted_rand_score as ARI, silhouette_score as SIL
from sentence_transformers import SentenceTransformer  # encodes text documents to 768D vectors
from sklearn.decomposition import PCA   # PCA uses SVD to reduce dimensionality of the feature space
from colorunittest import run_unittest
import plotly.graph_objects as go               # import graph object from plotly library

pd.set_option('max_rows', 5, 'max_columns', 40, 'max_colwidth', 100, 'precision', 2) # print format
df = pd.read_csv('movies.zip').fillna('').set_index('title') # load TMDB database

# **Task 1**

Your first task is to complete a UDF, which takes a TMDB dataframe `df` and a set of word tokens, then returns a masking list of Boolean (True/False) values indicating which movies in `df` contain any of these tokens in lower-cased `overview` description field. This will allow you to filter `df` to just ~100 movies that you will be clustering using hierarchical clustering ([HC](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html)) algorithm. When searching for tokens in `overview` attribute, you need to parse the latter using NLTK's `word_tokenize()` method with default parameters. For example, here is an output of `MakeMask(df[:10], {'century', 'is'})`:

    [True, True, False, True, True, False, False, True, False, False]


In [ ]:
# COMPLETE THIS CELL
def MakeMask(df=df, tok={'pet','pets','animal','animals','cat','dog','dogs'}) -> [bool]:
    '''Builds a masking list of Booleans, indicating which row of TMDB movie dataframe, df,
        contains any of the listed tokens in its (lower-cased) `overview` column, 
        which needs to be tokenized into words using the NLTK's `word_tokenize()`.
        If `overview` column doesn't exist, return an empty list.
    Inputs:
        df: TMDB movie dataframe with NxP shape, which contains `overview` column with movie description.
        tok: set of word tokens, which are searched for in the `overview` column
    Returns: a list of N Boolean values (True or False), where True indicates that `overview`
        contains a token from `tok` argument.    '''
    LbMask = []   # masking list
    # YOUR CODE HERE
    raise NotImplementedError()
    return LbMask

LbMask = MakeMask()  # define a mask on full movie dataframe
MakeMask(df[:10], {'century', 'is'})

Below is a set of tests for you and for autograder, which your implementation of the UDF should pass. If any test fails, spend the time to understand what the test is testing and why your function doesn't generate the same values. You can create a separate code cell and test your function just with the parameters specified in the test. 

In [ ]:
# RUN CELL TO TEST YOUR CODE
ae = npt.assert_equal
df1, df2 = df[:10], df[:1000]

@run_unittest
class Test_MakeMask(unittest.TestCase): # class with methods to test functionality of the JS() function
    def test00(self): ae(type(MakeMask(df1)), list)    # MakeMask must return a list
    def test01(self): ae(len(MakeMask(df1)), len(df1)) # returned list must have len(df_) elements
    def test02(self): ae(sum(MakeMask(df2)), 36)       # count of movies with default tokens
    def test03(self): ae(sum(MakeMask(df1, {'following'})), 2)
    def test04(self): ae(sum(MakeMask(df1, {'Following'})), 0)
    def test05(self): ae(sum(MakeMask(df1, {'is'})), 5)
    def test06(self): ae(MakeMask(df1), [False]*len(df1)) # Ensure Booleans are returned
    def test07(self): ae(MakeMask(df.drop('overview', axis=1)), []) # return [] if no `overvew` column
    def test08(self): ae(list(df2[MakeMask(df2)][:2].index), ['Alice in Wonderland', 'G-Force'])
    def test09(self): ae(list(df1[MakeMask(df1, {'is'})].index[[0,3]]), ['Avatar', 'John Carter'])
        

# **Task 2**

Next, the [`SBERT`](https://www.sbert.net/) object is defined as a sentence transformer with a pre-trained language model. Your task is to complete the `Encode` UDF to embed all movies in its `df` argument, assuming `overview` and `title` (set as index) attribute are in `df`. Read the UDF documentation carefully to properly implement this UDF. The output should be a dataframe with as many rows as there are movies in `df` argument and as many columns as `SBERT` model defines, i.e., 768 numeric columns in this case. This UDF embeds each movie in a 768-dimensional space. Here is the output of `dfEmb.iloc[:2,:3]`:

|title|0|1|2|
|-|-|-|-|
|Alice in Wonderland|0.0699525|-0.0560130|-0.0597558|
|G-Force|-0.1967611|0.1844813|0.5926098|

In [ ]:
SBERT = SentenceTransformer('paraphrase-albert-small-v2')  # load a pre-trained language model

In [ ]:
# COMPLETE THIS CELL
def Encode(df, SBERT=SBERT) -> pd.DataFrame():
    ''' Use SBERT model to encode title (set as index) and overview concatenated (in that order) via '. '.
    Wrap the resulting array as a dataframe with title as the row label/index.
    Inputs:
        df: TMDB dataframe
        SBERT: loaded sentence transformer model
    Returns: dataframe of encoded movie overviews into 768-dimensional vector space.       '''
    # YOUR CODE HERE
    raise NotImplementedError()
    return dfEmb

dfEmb = Encode(df[LbMask])  # define embedding dataframe for the whole TMDB dataframe
dfEmb.iloc[:2,:3]

In [ ]:
# RUN CELL TO TEST YOUR CODE
ae, aae = npt.assert_equal, npt.assert_almost_equal
df3 = df[LbMask][:10]

@run_unittest
class Test_Encode(unittest.TestCase): # class with methods to test functionality of the JS() function
    def test00(self): ae(type(Encode(df[:2])), pd.DataFrame) # Encode() must return a dataframe
    def test01(self): ae(list(Encode(df[:1]).index), ['Avatar']) # first row index is 'Avatar'
    def test02(self): ae(list(df[LbMask].index[:2]), ['Alice in Wonderland', 'G-Force'])
    def test03(self): ae(Encode(df[:1]).shape, (1, 768))  # vectors/rows must have 768 values
    def test04(self): aae(Encode(df[:1]).iloc[0,0], 0.093486995) # first coefficient for Avatar
    def test05(self): aae(Encode(df[:1]).iloc[0,-1], -0.7665929) # last coefficient for Avatar
    def test06(self): aae(Encode(df[:1]).T['Avatar'].to_list()[:4], [.0935,.1546,.2305,-.8012], decimal=4)
    def test07(self): aae(Encode(df[:1]).sum(1).values[0], -4.647999, decimal=5)
    def test08(self): aae(Encode(df[:10]).sum().values[:4], [ 0.529, -1.887,  2.252, -1.894], 3)
    def test09(self): aae(Encode(df[:10]).sum(1).values[:4], [-4.648, -4.432, -5.135, -4.616], 3)
    def test10(self): aae(Encode(df[:10]).sum().sum(), -45.63688, 5)
    def test11(self): aae(Encode(df[LbMask][:5]).sum().values[:5], [-1.74,-.43,.89,-2.94,-.04], 2)
    def test12(self): aae(Encode(df[LbMask][:5]).sum(1).values[:5], [-3.37,-6.14,-5.33,-5.2,-3.62], 2)
    def test13(self): aae(Encode(df[LbMask][:5]).sum().sum(), -23.65068, 4)


# **Task 3**


In this task you need to complete a UDF named `hc()`, which takes the movie embeddings, `dfEmb`, and two desired values of hyperparameters `k` (number of clusters) and [`linkage`](https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering) (type of distance used between clusters of points), and generates movie labels, which identify which cluster (from 0 to `k-1`) each movie belongs to. The predicted labels, `pY`, can be drawn from the `labels_` argument. Here is an output of `hc(dfEmb, k=2)[:10]`:

    array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
# COMPLETE THIS CELL
def hc(dfEmb, k=2, linkage='ward') -> np.array([bool]):
    '''Builds a trained/fitted hierarchical clustering model from HC() 
        with the specified k and linkage parameters.
    Inputs:
        dfEmb: NxP embeddings table with rows as vectors in P-dimensional space
        k: number of desired clusters
        linkage: type of linkage to use (ward, average, complete, single)
    Returns: a 1D array of N predicted cluster labels 
        (in range 0,1,...,k-1), one for each observation      '''
    # YOUR CODE HERE
    raise NotImplementedError()
    return pY   # predicted cluster labels for the HC algorithm fitted to dfEmb

hc(dfEmb, k=2)[:10]

In [ ]:
# RUN CELL TO TEST YOUR CODE
ae = npt.assert_equal
@run_unittest
class Test_hc(unittest.TestCase): # class with methods to test functionality of the JS() function
    def test00(self): ae(type(hc(dfEmb[:5])), np.ndarray) # must return numpy array type
    def test01(self): ae(hc(dfEmb[:5]), [0, 1, 0, 1, 1]) # default are 0-1 labels
    def test02(self): ae(hc(dfEmb[:5], 3), [2, 0, 1, 0, 0]) # 0-1-2 labels should be returned
    def test03(self): ae(hc(dfEmb[:5], 2, 'complete'), [0, 1, 0, 1, 1])
    def test04(self): ae(hc(dfEmb[:5], 2, 'single'), [1, 0, 0, 0, 0])
    def test05(self): ae(hc(dfEmb[:5], 2, 'average'), [1, 0, 0, 0, 0])
    def test06(self): ae(sum(hc(dfEmb[:5])), 3)  # count 1-labels
    def test07(self): ae(sum(hc(dfEmb[:100])), 54)
    def test08(self): ae(sum(hc(dfEmb[:100], 3)), 69)
    def test09(self): ae(sum(hc(dfEmb, 2)), 31)
    def test10(self): ae(sum(hc(dfEmb, 3)), 93)
    def test11(self): ae(sum(hc(dfEmb, 10)), 430)


# **Task 4**

Here you are tasked with implementing a `sil1()` UDF, which takes embedded movies dataframe, maximum number of clusters, `K`, and returns a dataframe of silhouette scores with the corresponding `k` values. The argument `ReturnBest` indicates whether to return dataframe row(s) with the best silhouette score(s) only. Here is an output of `sil1()`:

|.|SIL|k|
|-|-|-|
|0|0.0247463|2|
|1|0.0239634|3|

In [ ]:
# COMPLETE THIS CELL
pd.set_option('precision', 7) # print format

def sil1(dfEmb=dfEmb, K=3, ReturnBest=False) -> pd.DataFrame():
    '''Use hc() UDF to build and fit K-1 HC models with parameter k in the range 2,3,...,K.
    This UDF returns a dataframe with columns SIL (silhouette score) and k (number of clusters).
    If ReturnBest=True, then return (all) rows with the best SIL value.
    If K > number of observations, N, then reset K to N - 1
    Input: 
        dfEmb: NxP embedding matrix of P-dimensional row vectors
        K=3: max number of clusters
        ReturnBest: whether result should contain only the rows with the best SIL value
    Return: dataframe dfOut containing columns SIL and k (=2,3,..,K)        '''
    dfOut = pd.DataFrame([], columns=['SIL', 'k'])  # initialize output dataframe structure
    # YOUR CODE HERE
    raise NotImplementedError()
    return dfOut   # a dataframe with performance for K-1 models

sil1(K=20, ReturnBest=False)

In [ ]:
# RUN CELL TO TEST YOUR CODE
ae, aae = npt.assert_equal, npt.assert_almost_equal
dfEmb0 = pd.DataFrame([0,0,0,0])     # embedding dataframe with identical 1D points (i.e. a single cluster)
dfEmb1 = pd.DataFrame([0,0,0,1,1,1]) # embedding dataframe with 2 sets of 1D points (i.e. with 2 perfect clusters)
dfEmb2 = pd.DataFrame(list(range(5))*10) # embedding dataframe with 5 perfect clusters

@run_unittest
class Test_sil1(unittest.TestCase): # class with methods to test functionality of the JS() function
    def test00(self): ae(type(sil1()), pd.DataFrame)
    def test01(self): ae(sil1().shape, (2, 2))
    def test02(self): ae(sil1(dfEmb0, K=3).values, [[0,2],[0,3]])
    def test03(self): ae(sil1(dfEmb1, K=3).values, [[1,2],[.5,3]])
    def test04(self): ae(sil1(dfEmb2, K=3).values, [[.660063952985913,2],[.7122807017543861,3]])
    def test05(self): aae(sil1(dfEmb0, K=10, ReturnBest=True).values, [[0., 2.], [0., 3.]], 4)
    def test06(self): aae(sil1(dfEmb1, K=10, ReturnBest=True).values, [[1., 2.]], 4)
    def test08(self): ae(sil1(dfEmb2, K=10, ReturnBest=True).values, [[1., 5.]])
    def test07(self): ae(sil1(dfEmb0, K=100).values, [[0,2],[0,3]])
    def test09(self): ae(sil1(dfEmb1, K=100).values, [[1,2],[.5,3],[.5,4],[0,5]])
    def test10(self): aae(sil1(dfEmb2, K=100).values[:5,], [[.660,2],[.712,3],[.842,4],[1,5],[.8,6]],3)
    def test11(self): ae(sil1(dfEmb2, K=100).SIL.max(), 1)
    def test12(self): ae(sil1(dfEmb2, K=100).SIL.sum(), 5.214449917898194)
    def test13(self): ae(sil1(dfEmb2, 100, True).values, [[1,5]])
    def test14(self): aae(sil1(dfEmb, 3, False).values, [[.02474628,2],[.0239634,3]])
    def test15(self): aae(sil1(dfEmb, 3, True).values, [[.02474628,2]])
    def test16(self): aae(sil1(dfEmb, 30, True).values, [[.0375752,30]])


# **Task 5**

In this task you are building a `sil2()` UDF, which takes `K` (max clusters) and `linkages` (with a list of allowed linkage values) parameters, which `sil2` should search through to compute all possible clusters and their corresponding silhouette (SIL) scores on the embedded movie dataframe, `dfEmb`. As before, `ReturnBest` indicates whether rows with best SIL values should be returned in the output dataframe. Here is an output of `sil2(dfEmb, K=10, linkages=['average', 'complete', 'ward', 'single'], ReturnBest=True)`:

|.|SIL|k|link|
|-|-|-|-|
|0|0.157101|2|average|

In [ ]:
# COMPLETE THIS CELL
def sil2(dfEmb, K=2, linkages=['ward'], ReturnBest=False) -> pd.DataFrame():
    '''This UDF is similar to sil1(), but also allows for different values of linkage parameters.
    Essentially, it's either a double loop over k,linkage values or a call to hc() for each linkage value.
    This UDF returns a dataframe with columns SIL (silhouette score), k (number of clusters), and link (i.e. linkage).
    If ReturnBest=True, then return (all) rows with the best SIL value.
    If K > number of observations, N, then reset K to N - 1
    Input: 
        dfEmb: NxP embedding matrix of P-dimensional row vectors
        K=3: max number of clusters
        linkages: is a list of linkages (strings) to try: ward, complete, average, single 
        ReturnBest: whether result should contain only the rows with the best SIL value
    Return: dataframe dfOut containing columns SIL and k (=2,3,..,K)        '''
    dfOut = pd.DataFrame([], columns=['SIL', 'k', 'link'])    # initialize output dataframe structure
    if not isinstance(linkages, list): linkages = [linkages]  # ensures list structure
    # YOUR CODE HERE
    raise NotImplementedError()
    return dfOut   # a dataframe with performance for K-1 models for each linkage

dfGS = sil2(dfEmb, K=10, linkages=['average', 'complete', 'ward', 'single'], ReturnBest=True)
dfGS

In [ ]:
# RUN CELL TO TEST YOUR CODE
ae, aae = npt.assert_equal, npt.assert_almost_equal
dfEmb0 = pd.DataFrame([0,0,0,0])     # embedding dataframe with identical 1D points (i.e. a single cluster)
dfEmb1 = pd.DataFrame([0,0,0,1,1,1]) # embedding dataframe with 2 sets of 1D points (i.e. with 2 perfect clusters)
dfEmb2 = pd.DataFrame(list(range(5))*10) # embedding dataframe with 5 perfect clusters

@run_unittest
class Test_sil2(unittest.TestCase): # class with methods to test functionality of the JS() function
    def test00(self): ae(sil2(dfEmb0).values.tolist(), [[0, 2, 'ward']])
    def test01(self): ae(sil2(dfEmb0, K=3).values.tolist(), [[0, 2, 'ward'], [0, 3, 'ward']])
    def test02(self): ae(sil2(dfEmb0,2,['complete']).values.tolist(), [[0, 2, 'complete']])
    def test03(self): ae(sil2(dfEmb0, K=10).values.tolist(), [[0, 2, 'ward'], [0, 3, 'ward']])
    def test04(self): ae(sil2(dfEmb0, K=10, ReturnBest=True).values.tolist(), [[0, 2, 'ward'], [0, 3, 'ward']])
    def test05(self): ae(sil2(dfEmb1).values.tolist(), [[1, 2, 'ward']])
    def test06(self): ae(sil2(dfEmb1, 100, ['single'], True).values.tolist(), [[1, 2, 'single']])
    def test07(self): ae(sil2(dfEmb2).values.tolist(), [[0.660063952985913, 2, 'ward']])
    def test08(self): ae(sil2(dfEmb2, 10, ['average'], True).values.tolist(), [[1, 5, 'average']])
    def test09(self): ae(sil2(dfEmb2, 10, ['average','ward'], True).values.tolist(), [[1.0, 5, 'average'], [1.0, 5, 'ward']])
    def test10(self): ae(sil2(dfEmb, 10, ['ward'], True).values.tolist(), [[0.024746283888816833, 2, 'ward']])


# **Visualizations**

## **Hyperparameter Grid Search**

You do not need to code in sections below. These are for visualization purposes only. You can augment parameters to investigate different clustering performances.

In the plot below you are observing the performance of hyperparameter grid search, where all desired values of `k` are plotted for each linkage. Ward linkage has lowest SIL score and hence is plotted against a secondary y axis. The plot below suggests that all but Ward linkages perform best with `k=2` value, suggesting that there are two fairly distinct clusters, possibly relating to movies about cats and dogs (to be investigated further), since there were also the keywords in the `overview` texts. The poorer clustering is likely due to various other movie clusters relating to other (non cat or dog) movies, since we also pulled movies with animals and pets in the `overview` texts. 

In [ ]:
dfGS = sil2(dfEmb, K=20, linkages=['average', 'complete','ward','single'])
dfGS.set_index('k').groupby('link')['SIL'].plot(grid=True, figsize=[15,4], secondary_y=['ward'], legend=True, title='Silhouette score @k');

## **Scatterplot of Colored Clusters**

The next plot compresses all 768-dimensional movie vectors to just two dimensions using a principal component analysis (PCA) compression. The 2D movie points can now be plotted on screen. The resulting dataframe `dfPC12` has the same number of movies as there are in `df` and in `dfEmb` (and in the same order), but it has only two coordinate columns, `x` and `y`, and a column containing a cluster label.

In [ ]:
mPC12 = PCA(n_components=2).fit_transform(dfEmb)   # project 768-dim vectors to 2D space for plotting
dfPC12 = pd.DataFrame(mPC12, columns=['x','y'], index=dfEmb.index)
hcLabels = hc(dfEmb, k=3)
dfPC12['cluster'] = hcLabels # retrieve learnt cluster labels
dfPC12                       # contains new (x,y) coordinates and cluster labels

To indicate which cluster each $(x,y)$ point belongs to in the plot, the points are colored according to their cluster id (i.e., values from 0 to `k-1`). The following cell generates RGB (red, green, and blue) color strings, i.e., one color for each cluster.

In [ ]:
sPlotTtl = 'Clusters Identified by Hierarhichal Clusterning Algorithm'
LsPalette = [f'rgb({c[0]},{c[1]},{c[2]})' for c in sns.color_palette('bright', max(hcLabels)+1)]  # strings of RGB color values
vColors = np.array(LsPalette)[dfPC12.cluster]   # vector of colors (as RGB string) for each point 
vColors[:2]

Finally, the interactive scatter plot is plotted using the Plotly library. So, you can hover the mouse over each movie point to read its title. The `overview` descriptions are too long and would need to be wrapped with `<br>` HTML tags every 30-40 characters (which you can try as a separate exercise).

Note that the colored clusters appear to be concentrated, not randomly dispersed, which is what we desired. You can evaluate these points to decide whether these movie clusters make sense, i.e. contain "similar" movies (i.e. similar movie `title` and `overview` descriptions).

It seems that the blue movies are mostly about cats, the red movies are about dogs, and the green movies are about groups of animals. If you investigate these colored points, do you agree with this conclusion?

In [ ]:
sMovieGenres = [a + '<br> ' + b for a,b in zip(dfPC12.index, df[LbMask].tagline)] # point labels with title+genre
DMarkers = dict(size=5, line=dict(width=1, color=vColors), color=vColors)
goMargin = go.layout.Margin(l=0, r=0, b=0, t=0)
goS = go.Scatter(x=dfPC12.x, y=dfPC12.y, mode='markers', marker=DMarkers, text=sMovieGenres);
print(sPlotTtl)
goLayout = go.Layout(hovermode='closest', margin=goMargin, width=1000, 
                   height=300, xaxis={'title':'PC1'}, yaxis={'title':'PC2'});

fig = go.Figure(layout=goLayout)  # prepare a figure with specified layout
fig.add_trace(goS)                # add points to canvas